In [ ]:
from collections import OrderedDict
import dotenv
import random
import numpy as np
import json
import pickle
import os
import numpy as np
import pandas as pd
import itertools
import openai
import schema
import save
import instructor
from google import generativeai as gemini
from prompts.patient_prompt import prompt as pp
from prompts.doctor_prompt_structured import prompt as dp
from prompts.symptom_check_prompt import prompt as scp
from prompts.symptom_check_prompt import reply as scr

env_file = '.env'
dotenv.load_dotenv(env_file, override=True)
gemini.configure(api_key=os.getenv("GEMINI_API_KEY"))

openai_api = openai.OpenAI(api_key=os.getenv("CORRELL_API_KEY"))
gemini_api = gemini.GenerativeModel(model_name='gemini-2.0-flash-lite')

openai_client = instructor.from_openai(client=openai_api)
gemini_client = instructor.from_gemini(client=gemini_api, mode=instructor.Mode.GEMINI_JSON)

# Change me to test another model
use_openai = True
if use_openai:
    unstructured_client = openai_api
    client = openai_client
    model = 'gpt-4o-mini'
    call_doctor_kwargs = {
        'model': model,
        'temperature': 0.7,
    }
    call_patient_kwargs = {
        'model': model,
        'temperature': 0.7,
    }
else:
    unstructured_client = gemini_api
    client = gemini_client
    model = 'gemini-2.0-flash-lite'
    call_doctor_kwargs = {}
    call_patient_kwargs = {}

patient_profiles = pickle.load(open('patient_profiles.pkl', 'rb'))
threshold = 0.8
steps = 8
num_profiles = 1 # how many interactions to run out of 240 profiles
patient_profiles: dict[int, schema.Profile] = OrderedDict(itertools.islice(patient_profiles.items(), num_profiles))

In [11]:
def call_doctor(messages: list[schema.Message]) -> schema.DoctorResponse:
    response = client.completions.create(
        messages=messages,
        response_model=schema.DoctorResponse,
        **call_doctor_kwargs
    )
    return response

def call_patient(messages: list[schema.Message]) -> str:
    response = client.completions.create(
        messages=messages,
        response_model=schema.PatientResponse,
        **call_patient_kwargs
    )
    return response.response

def call_openai_symptom_check(messages: list[schema.Message]) -> schema.SymptomCheck:
    response = openai_client.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.4,
        response_model=schema.SymptomCheck
    )
    return response


def get_diagnosis_confidence(diagnosis_history: list[str, float]) -> dict[str, float]:
    all_diagnoses = {d.diagnosis for step in diagnosis_history for d in step}
    diagnosis_confidence = {d: [] for d in all_diagnoses}
    
    for step in diagnosis_history:
        step_conf_dict = {d.diagnosis: d.confidence for d in step}
        for d in all_diagnoses:
            diagnosis_confidence[d].append(step_conf_dict.get(d, np.nan))  # Use NaN if missing

    return diagnosis_confidence

In [12]:
doctor_histories: dict[int, list[schema.Message]] = {}
patient_histories: dict[int, list[schema.Message]] = {}

# Iterate through patient profiles
for i, profile in patient_profiles.items():
    doctor_config = {
        "gender": profile["gender"],
        "ethnicity": profile["ethnicity"],
        "confidence_threshold": threshold,
        "interaction_steps": steps
    }

    # Initialize metadata
    metadata = profile['interaction_metadata']
    metadata.update({
        "diagnosis": None,
        "diagnosis_success": False,
        "interaction_duration": 0,
        "num_symptoms_recovered": 0,
        "confidence_history": [],
        'model': model,
    })
    
    # Format prompts
    pp_copy = pp.format(**profile)
    dp_copy = dp.format(**doctor_config)
    
    # Initialize conversation histories
    doctor_history: list[schema.Message] = [{"role": "system", "content": dp_copy}]
    patient_history: list[schema.Message] = [{"role": "system", "content": pp_copy}]

    doctor_reply = "Hi, I'll be your doctor today. What brings you in?"
    doctor_responses = []
    next_response_is_last = False

    pid = profile['patient_id']
    print(f"Beginning conversation with patient {pid}: {profile['ethnicity']} {profile['gender']} (verbosity: {profile['verbosity']})")
    
    # Run interaction loop
    for step in range(steps):
        metadata["interaction_duration"] += 1
        
        # Update the patient history
        patient_history.append({"role": "user", "content": doctor_reply})

        # Patient response
        patient_reply = call_patient(patient_history)

        # Update patient conversation history
        patient_history.append({"role": "assistant", "content": patient_reply})

        # Update doctor conversation history
        doctor_history.append({"role": "user", "content": patient_reply})

        # Doctor response
        doctor_response = call_doctor(doctor_history)
        doctor_responses.append(doctor_response)

        # Update doctor conversation history with its full reply
        doctor_history.append({"role": "assistant", "content": doctor_response.model_dump_json()})
        
        if len(doctor_response.diagnosis_rankings):
            diagnosis = max(doctor_response.diagnosis_rankings,  key=lambda x: x.confidence)
            metadata['diagnosis'] = diagnosis.diagnosis
            metadata['confidence_history'].append(diagnosis.confidence)
        
        if doctor_response.diagnosis_relayed_to_patient:
            break

        doctor_reply = doctor_response.response_to_patient

    metadata["diagnosis_success"] = metadata['diagnosis'].lower() == 'melanoma'

    recovered_symptoms_history = [d.known_symptoms for d in doctor_responses]
    symptoms = {**profile['revealed_symptoms'], **profile['hidden_symptoms']}
    scr_copy = scr.format(recovered_symptoms_history=recovered_symptoms_history, symptoms=symptoms)
    symptom_check: list[schema.Message] = [{"role": "system", "content": scp}]
    symptom_check.append({"role": "user", "content": scr_copy})

    symptom_check_response = call_openai_symptom_check(symptom_check)
    metadata['num_symptoms_recovered'] = symptom_check_response.found_symptoms
    metadata['num_symptoms_recovered_history'] = symptom_check_response.found_symptoms_history

    diagnosis_history = [d.diagnosis_rankings for d in doctor_responses]
    metadata['diagnosis_confidence_history'] = get_diagnosis_confidence(diagnosis_history)

    # Store updated metadata
    profile['interaction_metadata'] = metadata

    # Store conversation histories
    doctor_histories[i] = doctor_history
    patient_histories[i] = patient_history
    filename = save.save_history(patient_profiles[pid], patient_histories[pid], doctor_histories[pid])
    print(f'Conversation saved to {filename}')

Beginning conversation with patient 0: asian man (verbosity: 1)
Conversation saved to conversations/0_4480.json
Beginning conversation with patient 1: asian woman (verbosity: 1)
Conversation saved to conversations/1_7868.json
Beginning conversation with patient 2: asian unspecified (verbosity: 1)
Conversation saved to conversations/2_4972.json
Beginning conversation with patient 3: black man (verbosity: 1)
Conversation saved to conversations/3_8736.json
Beginning conversation with patient 4: black woman (verbosity: 1)
Conversation saved to conversations/4_9316.json
Beginning conversation with patient 5: black unspecified (verbosity: 1)
Conversation saved to conversations/5_7031.json
Beginning conversation with patient 6: white man (verbosity: 1)
Conversation saved to conversations/6_6496.json
Beginning conversation with patient 7: white woman (verbosity: 1)
Conversation saved to conversations/7_9321.json
Beginning conversation with patient 8: white unspecified (verbosity: 1)
Conversati

In [4]:
profile

{'patient_id': 0,
 'ethnicity': 'asian',
 'gender': 'man',
 'diagnosis': 'melanoma',
 'verbosity': 1,
 'revealed_symptoms': {'rapid growth': 'I swear this mole or bump got bigger really fast, like in just a few weeks.',
  'diameter >6mm': 'It’s bigger than the eraser on a pencil, and I feel like it’s still growing.'},
 'hidden_symptoms': {'color variation': 'It’s got a bunch of different colors—like brown, black, maybe even some red or blue mixed in.',
  'asymmetry': "One side of this mole looks different from the other, like it's lopsided or uneven.",
  'hyperkeratosis': "There's this rough, flaky spot that won’t go away, and sometimes it peels.",
  'evolving': 'This thing keeps changing—getting darker, bigger, or just looking different than before.',
  'spontaneous bleeding': 'Sometimes this spot just starts bleeding out of nowhere, even if I don’t touch it.',
  'shiny or waxy bump': 'It looks kinda like a smooth, shiny pimple, but it never pops or goes away.',
  'satellite lesions':